<a href="https://colab.research.google.com/github/jjikku/OptiGene-Algo/blob/main/ICT_Group_2_ML_AI_Algo_Trading_LSTM_based_on_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specify the exact path to the CSV file
file_path = "/content/drive/My Drive/NIfty_Dataset/NIFTY_before_scaling_and_outlier_clipping_8_weeks.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.columns.tolist()
df.head()

Mounted at /content/drive


,Date/Time,Ticker,Open,High,Low,Close,INDIAVIX,ATM_OI,ATM_Call_Volume,ATM_Put_Volume,...,Bollinger_High_15m,Bollinger_Mid_15m,Bollinger_Low_15m,MACD_Hist_15m,CE_Buys,PE_Buys,CE_Confidence_Score,PE_Confidence_Score,CE_Buys_conf,PE_Buys_conf
0,2021-04-09 09:35:00,NIFTY,14845.5,14860.5,14845.5,14855.5,20.65,446025.0,75750.0,75075.0,...,14873.984213,14845.353333,14816.722453,-0.637034,0,0,1.460000,0.000000,0,0
1,2021-04-09 09:36:00,NIFTY,14855.6,14867.2,14855.6,14865.1,20.57,446025.0,87525.0,78675.0,...,14873.623201,14848.746667,14823.870132,0.676582,0,0,0.333667,0.224000,0,0
2,2021-04-09 09:37:00,NIFTY,14865.2,14870.7,14858.6,14866.5,20.59,446025.0,86100.0,54825.0,...,14872.164921,14851.986667,14831.808412,1.301171,0,0,0.413000,0.239667,0,0
3,2021-04-09 09:38:00,NIFTY,14867.0,14869.1,14859.2,14859.2,20.58,506175.0,50175.0,33900.0,...,14870.279846,14853.993333,14837.706821,0.692525,0,0,1.864000,0.000000,1,0
4,2021-04-09 09:39:00,NIFTY,14858.0,14868.1,14856.5,14866.0,20.57,506175.0,51675.0,48450.0,...,14871.201113,14855.726667,14840.252220,0.859931,0,0,1.383000,0.000000,0,0


In [2]:
df_to_LSTM  = df.drop(['Ticker','CE_Buys','PE_Buys','CE_Buys_conf','PE_Buys_conf'], axis=1)

In [3]:
# Print bottom 5 rows with the lowest Confidence_Score_Pred
print("Bottom 5 rows with the lowest Confidence_Score:")
print(df_to_LSTM.nsmallest(50, 'CE_Confidence_Score'))


Bottom 5 rows with the lowest Confidence_Score:
                 Date/Time     Open     High      Low    Close  INDIAVIX  \
6605   2021-10-08 12:55:00  17890.0  17891.0  17886.3  17888.5     15.77   
1178   2021-04-15 11:25:00  14371.0  14377.6  14371.0  14372.7     21.47   
783    2021-04-13 10:46:00  14375.5  14385.6  14375.5  14385.6     21.53   
11730  2022-01-07 15:24:00  17828.3  17829.5  17823.9  17826.7     17.60   
6961   2021-10-11 12:56:00  18030.3  18032.0  18028.9  18030.5     16.06   
10710  2021-12-08 10:16:00  17411.0  17411.0  17402.0  17402.0     17.62   
10428  2021-12-07 11:30:00  17172.5  17174.0  17171.0  17171.5     18.73   
2425   2021-05-11 14:25:00  14841.1  14841.3  14835.9  14835.9     19.85   
1461   2021-05-07 10:12:00  14837.3  14837.7  14833.4  14833.4     21.36   
10705  2021-12-08 10:11:00  17408.4  17414.8  17407.6  17410.7     17.60   
11064  2021-12-09 10:14:00  17415.5  17415.5  17408.7  17411.8     17.33   
5243   2021-10-04 13:57:00  17709.2  177

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# List of columns to exclude from normalization (binary target columns + 'Date/Time')
columns_to_exclude = ['Date/Time']

# Get the list of columns to normalize (all columns except the ones to exclude)
columns_to_normalize = [col for col in df_to_LSTM.columns if col not in columns_to_exclude]

# Initialize the feature scaler
feature_scaler = MinMaxScaler()

# Fit and transform the features
df_to_LSTM[columns_to_normalize] = feature_scaler.fit_transform(df_to_LSTM[columns_to_normalize])

# Initialize the target scaler
target_scaler = MinMaxScaler()

# Assuming your target columns are 'CE_Confidence_Score' and 'PE_Confidence_Score'
target_columns = ['CE_Confidence_Score', 'PE_Confidence_Score']
df_to_LSTM[target_columns] = target_scaler.fit_transform(df_to_LSTM[target_columns])


df_to_LSTM['Date/Time'] = df_to_LSTM['Date/Time']



In [5]:
from joblib import dump

# save the scaler in your Drive
dump(target_scaler, '/content/drive/My Drive/OptiGene Model/target_scaler.joblib')


['/content/drive/My Drive/OptiGene Model/target_scaler.joblib']

In [6]:
# # Get the explained variance ratio of the components
# explained_variance_ratio = pca.explained_variance_ratio_
# print(explained_variance_ratio)


In [7]:
# # Get the PCA components (loadings)
# pca_components = pca.components_
# df_components = pd.DataFrame(pca_components, columns=columns_to_normalize)
# print(df_components)


In [8]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Create a DataFrame for the PCA components
# df_components = pd.DataFrame(pca.components_, columns=columns_to_normalize)

# # Plot a heatmap
# plt.figure(figsize=(15, 10))
# sns.heatmap(df_components, cmap='viridis', annot=False)
# plt.title('PCA Component Loadings')
# plt.ylabel('Principal Component')
# plt.xlabel('Feature')
# plt.xticks(rotation=90)  # Rotate feature names for better readability
# plt.show()


In [9]:
from sklearn.model_selection import train_test_split

# Assuming df_to_LSTM is your DataFrame
# Convert 'Date/Time' to datetime if it's not already
df_to_LSTM['Date/Time'] = pd.to_datetime(df_to_LSTM['Date/Time'])

# Sort the data by 'Date/Time' just in case
df_to_LSTM.sort_values('Date/Time', inplace=True)

# Drop the 'Date/Time' column if it won't be used as a feature
df_to_LSTM.drop('Date/Time', axis=1, inplace=True)

# Split data into features and target
X = df_to_LSTM.drop(['CE_Confidence_Score','PE_Confidence_Score'], axis=1)
y = df_to_LSTM[['CE_Confidence_Score','PE_Confidence_Score']]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [10]:
X.columns.tolist()

['Open',
 'High',
 'Low',
 'Close',
 'INDIAVIX',
 'ATM_OI',
 'ATM_Call_Volume',
 'ATM_Put_Volume',
 'ATM_Gamma',
 'ATM_Vega',
 'ITM1_OI',
 'ITM1_Call_Volume',
 'ITM1_Put_Volume',
 'ITM1_Gamma',
 'ITM1_Vega',
 'OTM1_OI',
 'OTM1_Call_Volume',
 'OTM1_Put_Volume',
 'OTM1_Gamma',
 'OTM1_Vega',
 'ITM2_OI',
 'ITM2_Call_Volume',
 'ITM2_Put_Volume',
 'ITM2_Gamma',
 'ITM2_Vega',
 'OTM2_OI',
 'OTM2_Call_Volume',
 'OTM2_Put_Volume',
 'OTM2_Gamma',
 'OTM2_Vega',
 'ITM3_OI',
 'ITM3_Call_Volume',
 'ITM3_Put_Volume',
 'ITM3_Gamma',
 'ITM3_Vega',
 'OTM3_OI',
 'OTM3_Call_Volume',
 'OTM3_Put_Volume',
 'OTM3_Gamma',
 'OTM3_Vega',
 'ITM4_OI',
 'ITM4_Call_Volume',
 'ITM4_Put_Volume',
 'ITM4_Gamma',
 'ITM4_Vega',
 'OTM4_OI',
 'OTM4_Call_Volume',
 'OTM4_Put_Volume',
 'OTM4_Gamma',
 'OTM4_Vega',
 'ITM5_OI',
 'ITM5_Call_Volume',
 'ITM5_Put_Volume',
 'ITM5_Gamma',
 'ITM5_Vega',
 'OTM5_OI',
 'OTM5_Call_Volume',
 'OTM5_Put_Volume',
 'OTM5_Gamma',
 'OTM5_Vega',
 'PCR',
 'ATM_PE_OI',
 'ATM_PE_Gamma',
 'ATM_PE_Vega',

In [11]:
# import numpy as np

# def create_sequences(input_data, output_data, time_steps):
#     Xs, ys = [], []
#     for i in range(len(input_data) - time_steps):
#         Xs.append(input_data.iloc[i:(i + time_steps)].values)
#         ys.append(output_data.iloc[i + time_steps])
#     return np.array(Xs), np.array(ys)

# # Define the time step
# time_steps = 20  # This need to be be fine tuned

# # Create sequences from the training and testing data
# X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
# X_test_seq, y_test_seq = create_sequences(X_test, y_test, time_steps)


In [12]:
df_to_LSTM.isna().sum()

Open                   0
High                   0
Low                    0
Close                  0
INDIAVIX               0
                      ..
Bollinger_Mid_15m      0
Bollinger_Low_15m      0
MACD_Hist_15m          0
CE_Confidence_Score    0
PE_Confidence_Score    0
Length: 147, dtype: int64

In [13]:
import numpy as np

def create_sequences_rolling_window(input_data, output_data, time_steps):
    Xs, ys = [], []
    # Adjust the range to allow for the rolling window
    for i in range(len(input_data) - time_steps + 1):
        Xs.append(input_data.iloc[i:(i + time_steps)].values)
        ys.append(output_data.iloc[i + time_steps - 1])
    return np.array(Xs), np.array(ys)

# Define the time step
time_steps = 20  # Fine-tune this as needed

# Create sequences from the training and testing data using the rolling window approach
X_train_seq, y_train_seq = create_sequences_rolling_window(X_train, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences_rolling_window(X_test, y_test, time_steps)


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define the input shape for the LSTM model
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])

# Build the LSTM model
model = Sequential()

# First LSTM layer with Dropout and Batch Normalization
model.add(LSTM(264, return_sequences=True, input_shape=input_shape, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Second LSTM layer with Dropout and Batch Normalization
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Second LSTM layer with Dropout and Batch Normalization
model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Third LSTM layer
model.add(LSTM(32, return_sequences=False, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Dense layer for output
model.add(Dense(2, activation='linear'))  # Output layer for 2 predictions

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

# Fit the model
model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, validation_split=0.1, verbose=1, callbacks=[early_stopping])

# Evaluate the model
evaluation = model.evaluate(X_test_seq, y_test_seq)
print(f"Evaluation - Loss: {evaluation[0]}, RMSE: {evaluation[1]}")


Epoch 1/50
296/296 [==============================] - 12s 16ms/step - loss: 0.3669 - root_mean_squared_error: 0.6058 - val_loss: 0.0248 - val_root_mean_squared_error: 0.1576
Epoch 2/50
296/296 [==============================] - 4s 12ms/step - loss: 0.1122 - root_mean_squared_error: 0.3350 - val_loss: 0.0228 - val_root_mean_squared_error: 0.1510
Epoch 3/50
296/296 [==============================] - 4s 12ms/step - loss: 0.0464 - root_mean_squared_error: 0.2153 - val_loss: 0.0205 - val_root_mean_squared_error: 0.1432
Epoch 4/50
296/296 [==============================] - 3s 12ms/step - loss: 0.0241 - root_mean_squared_error: 0.1553 - val_loss: 0.0208 - val_root_mean_squared_error: 0.1444
Epoch 5/50
296/296 [==============================] - 3s 12ms/step - loss: 0.0167 - root_mean_squared_error: 0.1291 - val_loss: 0.0205 - val_root_mean_squared_error: 0.1431
Epoch 6/50
296/296 [==============================] - 4s 12ms/step - loss: 0.0144 - root_mean_squared_error: 0.1200 - val_loss: 0.0204

In [33]:
model_save_path = '/content/drive/My Drive/OptiGene Model/my_lstm_Gen5_GA_model'
model.save(model_save_path)


In [ ]:
# !pip install deap

In [ ]:
# import random
# from deap import base, creator, tools, algorithms
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
# from tensorflow.keras.metrics import RootMeanSquaredError
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# import numpy as np
# import tensorflow as tf

# # Expanded hyperparameters
# params = {
#     'num_units': [264],  # Number of LSTM units
#     'dropout_rate': [0.1],  # Dropout rate
#     'batch_size': [32],  # Batch size
#     'learning_rate': [0.0001, 0.0005, 0.001],  # Learning rate
#     'num_layers': [2,3],  # Number of LSTM layers
#     'activation_function': ['tanh', 'relu', 'leaky_relu'],
#     'recurrent_activation': ['sigmoid', 'tanh']
# }

# # Fitness function to evaluate a set of hyperparameters
# def evaluate(individual):
#     # Unpack and cast parameters as needed
#     num_units = int(individual[0])  # Convert to integer
#     dropout_rate = max(0.0, min(individual[1], 0.999))  # Clamp to [0, 0.999)
#     batch_size = int(individual[2]) # Convert to integer
#     learning_rate = individual[3]  # Float, no need to convert
#     num_layers = int(individual[4]) # Convert to integer
#     activation_function = individual[5]  # Activation function
#     recurrent_activation = individual[6]  # Recurrent activation
#     early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#     # Build the LSTM model with the specified parameters
#     model = Sequential()
#     for i in range(num_layers):
#         if i == 0:
#             model.add(LSTM(num_units, activation=activation_function, recurrent_activation=recurrent_activation, input_shape=(time_steps, X_train_seq.shape[2]), return_sequences=num_layers > 1))
#         else:
#             model.add(LSTM(num_units, activation=activation_function, recurrent_activation=recurrent_activation, return_sequences=i < num_layers - 1))
#         model.add(Dropout(dropout_rate))

#     # Add the output layer
#     model.add(Dense(2, activation='linear'))

#     # Compile the model
#     optimizer = Adam(learning_rate=learning_rate)
#     model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

#     try:
#         model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=batch_size, validation_split=0.1, verbose=1, callbacks=[early_stopping])
#         loss, rmse = model.evaluate(X_test_seq, y_test_seq, verbose=1)
#     except Exception as e:
#         # Handle any exceptions during training/evaluation
#         print(f"Training/evaluation failed with exception: {e}")
#         return np.inf, None  # Return a high RMSE value to penalize this model

#     # Check for NaN values in RMSE and loss
#     if np.isnan(rmse) or np.isnan(loss):
#         print(f"Individual: {individual}, RMSE: NaN, Loss: NaN")
#         return np.inf, None  # Return a high RMSE value for NaN outcomes

#     # Print and return the RMSE and model for valid outcomes
#     print(f"Individual: {individual}, RMSE: {rmse}, Loss: {loss}")
#     return rmse, model

# # Genetic Algorithm setup
# creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMin)

# toolbox = base.Toolbox()
# toolbox.register("attr_recurrent_activation", random.choice, params['recurrent_activation'])
# toolbox.register("attr_activation_function", random.choice, params['activation_function'])
# toolbox.register("attr_num_units", random.choice, params['num_units'])
# toolbox.register("attr_dropout_rate", random.choice, params['dropout_rate'])
# toolbox.register("attr_batch_size", random.choice, params['batch_size'])
# toolbox.register("attr_learning_rate", random.choice, params['learning_rate'])
# toolbox.register("attr_num_layers", random.choice, params['num_layers'])
# toolbox.register("individual", tools.initCycle, creator.Individual,
#                  (toolbox.attr_num_units, toolbox.attr_dropout_rate, toolbox.attr_batch_size, toolbox.attr_learning_rate, toolbox.attr_num_layers, toolbox.attr_activation_function, toolbox.attr_recurrent_activation), n=1)


# toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("evaluate", evaluate)

# # Mutation function with clamping and handling string-based parameters
# def clamp_mutation(individual, low, up, indpb, string_params_indices):
#     for i in range(len(individual)):
#         if random.random() < indpb:
#             if i in string_params_indices:
#                 # Randomly select a new value for string-based parameters
#                 individual[i] = random.choice(params[list(params.keys())[i]])
#             else:
#                 # Numeric parameters
#                 individual[i] += random.gauss(0, 1)
#                 individual[i] = min(max(individual[i], low[i]), up[i])
#     return individual,



# low = [min(params['num_units']), 0.0, min(params['batch_size']), min(params['learning_rate']), min(params['num_layers'])]
# up = [max(params['num_units']), 1.0, max(params['batch_size']), max(params['learning_rate']), max(params['num_layers']), 'leaky_relu', 'hard_sigmoid']

# string_params_indices = [5, 6]  # Indices of string-based parameters in the individual
# # Mutation registration with correct bounds

# toolbox.register("mutate", clamp_mutation, low=low, up=up, indpb=0.2, string_params_indices=string_params_indices)
# toolbox.register("mate", tools.cxTwoPoint)
# toolbox.register("select", tools.selTournament, tournsize=3)

# # Statistics
# stats = tools.Statistics(key=lambda ind: ind.fitness.values)
# stats.register("min", np.min)
# stats.register("max", np.max)
# stats.register("avg", np.mean)
# stats.register("std", np.std)

# # Run the genetic algorithm
# population = toolbox.population(n=50)
# n_generation = 10

# model_dict = {}  # Dictionary to keep track of models

# for gen in range(n_generation):
#     offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
#     fits = toolbox.map(toolbox.evaluate, offspring)
#     for fit, ind in zip(fits, offspring):
#         ind.fitness.values = (fit[0],)  # Set fitness value
#         model_dict[str(ind)] = fit[1]  # Map individual to its model
#     population = toolbox.select(offspring, k=len(population))
#     # Gather and print statistics
#     record = stats.compile(population)
#     print(f"Generation {gen}: Min {record['min']}, Max {record['max']}, Avg {record['avg']}, Std {record['std']}")

#     # Print best individual of the generation
#     best_ind = tools.selBest(population, k=1)[0]
#     print(f"Best Individual of Generation {gen}: {best_ind}, RMSE: {best_ind.fitness.values[0]}")

# top10 = tools.selBest(population, k=10)
# best_ind = tools.selBest(population, k=1)[0]
# best_model = model_dict[str(best_ind)]

# # Save the best model
# model_save_path = '/content/drive/My Drive/OptiGene Model/my_lstm_GA_tuned_model_1'
# best_model.save(model_save_path)
# print(f"Best model saved to {model_save_path}")


In [ ]:
# import pandas as pd

# # Assuming top10 contains your top 10 individuals
# # And params dict has the same structure as provided in your code

# # Function to extract parameters from an individual
# def extract_params(individual):
#     return {
#         'num_units': int(individual[0]),
#         'dropout_rate': individual[1],
#         'batch_size': int(individual[2]),
#         'learning_rate': individual[3],
#         'num_layers': int(individual[4])
#     }

# # Extract parameters from the top individuals
# top_individuals_params = [extract_params(ind) for ind in top10]

# # Create a DataFrame
# df_params = pd.DataFrame(top_individuals_params)

# # Save to CSV
# df_params.to_csv('/content/drive/My Drive/NIfty_Dataset/best_model_parameters.csv', index=False)

# print("Best model parameters saved to 'best_model_parameters.csv'")


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import RootMeanSquaredError

# # Assuming best_hyperparams is a dictionary containing the best hyperparameters
# # Example: {'num_units': 50, 'dropout_rate': 0.2, 'batch_size': 32, 'learning_rate': 0.01, 'num_layers': 3, ...}

# model = Sequential()

# # Dynamically add LSTM layers based on the number of layers in best_hyperparams
# for i in range(best_hyperparams['num_layers']):
#     # For first layer, add input_shape
#     if i == 0:
#         model.add(LSTM(best_hyperparams['num_units'], activation='tanh', recurrent_activation='tanh', input_shape=(time_steps, X_train_seq.shape[2]), return_sequences=True if best_hyperparams['num_layers'] > 1 else False))
#     else:
#         # For last layer, do not return sequences
#         model.add(LSTM(best_hyperparams['num_units'], activation='tanh', recurrent_activation='tanh', return_sequences=i < best_hyperparams['num_layers'] - 1))

#     model.add(Dropout(best_hyperparams['dropout_rate']))

# # Add the output layer
# model.add(Dense(1, activation='tanh'))

# optimizer = Adam(learning_rate=best_hyperparams['learning_rate'])
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

# # Train the model
# model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=best_hyperparams['batch_size'], validation_split=0.1, verbose=1)

# # Save the model
# model.save("best_lstm_model.h5")


In [31]:
# Generate predictions for the test set
predictions = model.predict(X_test_seq)

print(f"Length of X_test_seq: {len(X_test_seq)}")
print(f"Length of predictions: {len(predictions)}")


82/82 [==============================] - 1s 8ms/step
Length of X_test_seq: 2613
Length of predictions: 2613


In [ ]:

# Calculate the start index for the test set in the original dataset
test_start_index = len(df_to_LSTM) - len(X_test)

# Calculate the start index for aligned 'Date/Time' values
# Since each prediction corresponds to the end of a sequence, we need to offset by time_steps
aligned_date_time_start_index = test_start_index + time_steps

# Align 'Date/Time' with the predictions
# The number of predictions is equal to the number of sequences in the test set
# aligned_date_time = df['Date/Time'].iloc[aligned_date_time_start_index:aligned_date_time_start_index + len(predictions)].reset_index(drop=True)
# Adjust the end index for slicing 'aligned_date_time'
aligned_date_time = df['Date/Time'].iloc[aligned_date_time_start_index:aligned_date_time_start_index + len(predictions) - 1].reset_index(drop=True)

print(f"Start index for test data: {test_start_index}")
print(f"Start index for aligned dates: {aligned_date_time_start_index}")
print(f"Length of df['Date/Time']: {len(df['Date/Time'])}")
print(f"Length of predictions: {len(predictions)}")
print(f"Sliced length of aligned dates: {len(aligned_date_time)}")

predictions_aligned = predictions[:-1]
assert len(aligned_date_time) == len(predictions_aligned), f"Alignment mismatch: {len(aligned_date_time)} dates aligned with {len(predictions_aligned)} predictions"

# # Ensure alignment
# assert len(aligned_date_time) == len(predictions), f"Alignment mismatch: {len(aligned_date_time)} dates aligned with {len(predictions)} predictions"

# Inverse scale Target
inverse_predictions = target_scaler.inverse_transform(predictions_aligned)

# Convert predictions to a DataFrame
df_predictions = pd.DataFrame({
    'CE_Confidence_Score_Pred': inverse_predictions[:, 0],
    'PE_Confidence_Score_Pred': inverse_predictions[:, 1],
    'Date/Time': aligned_date_time
})

# # Add the aligned 'Date/Time' to the predictions DataFrame
df_predictions['Date/Time'] = aligned_date_time

# Now df_predictions contains the predictions and the actual values
print(df_predictions)

In [ ]:

print(df_predictions)

In [ ]:
import pandas as pd

# Assuming df_predictions is your DataFrame
# Replace df_predictions with your actual DataFrame variable name

# Find maximum and minimum of CE_Confidence_Score_Pred
max_ce_pred = df_predictions['CE_Confidence_Score_Pred'].max()
min_ce_pred = df_predictions['CE_Confidence_Score_Pred'].min()

# Find maximum and minimum of PE_Confidence_Score_Pred
max_pe_pred = df_predictions['PE_Confidence_Score_Pred'].max()
min_pe_pred = df_predictions['PE_Confidence_Score_Pred'].min()

# Print the results
print("Maximum CE Confidence Score Prediction:", max_ce_pred)
print("Minimum CE Confidence Score Prediction:", min_ce_pred)
print("Maximum PE Confidence Score Prediction:", max_pe_pred)
print("Minimum PE Confidence Score Prediction:", min_pe_pred)


In [ ]:
# Normalize CE Confidence Score to 0-100 range
df_predictions['Normalized_CE_Confidence_Score'] = 100 * (df_predictions['CE_Confidence_Score_Pred'] - min_ce_pred) / (max_ce_pred - min_ce_pred)

# Normalize PE Confidence Score to 0-100 range
df_predictions['Normalized_PE_Confidence_Score'] = 100 * (df_predictions['PE_Confidence_Score_Pred'] - min_pe_pred) / (max_pe_pred - min_pe_pred)

# Check the result
print(df_predictions[['CE_Confidence_Score_Pred', 'PE_Confidence_Score_Pred', 'Normalized_CE_Confidence_Score', 'Normalized_PE_Confidence_Score']])

# Calculate the difference between normalized scores
df_predictions['Confidence_Score_Difference'] = df_predictions['Normalized_CE_Confidence_Score'] - df_predictions['Normalized_PE_Confidence_Score']

# Check the maximum and minimum of the difference
max_difference = df_predictions['Confidence_Score_Difference'].max()
min_difference = df_predictions['Confidence_Score_Difference'].min()

print("Maximum Difference:", max_difference)
print("Minimum Difference:", min_difference)


In [ ]:
# Apply a threshold to convert probabilities to binary predictions
threshold = 70

df_predictions['PE_Buys_Pred'] = ((df_predictions['Confidence_Score_Difference']) < -1 * threshold).astype(int)

# Filter rows where either CE_Buys_Pred_Class or PE_Buys_Pred_Class is 1
genuine_predictions_pe = df_predictions[(df_predictions['PE_Buys_Pred'] == 1)]

# Print the rows with genuine predictions
print(genuine_predictions_pe)



In [ ]:
genuine_predictions_pe.head(60)

In [ ]:
# Apply a threshold to convert probabilities to binary predictions
threshold = 75

df_predictions['CE_Buys_Pred'] = ((df_predictions['Confidence_Score_Difference']) > threshold).astype(int)


# Filter rows where either CE_Buys_Pred_Class or PE_Buys_Pred_Class is 1
genuine_predictions_ce = df_predictions[(df_predictions['CE_Buys_Pred'] == 1)]

# Print the rows with genuine predictions
print(genuine_predictions_ce)



In [ ]:
genuine_predictions_ce.head(50)